# PrioritizeMe

Aplikacja do priorytetowego planowania zadań to narzędzie, które pomoże efektywnie zarządzać czasem i zadaniami. Dzięki niej możesz tworzyć zadania, przypisywać im tytuły oraz opisy, a następnie nadawać im priorytety w zależności od ich ważności. 

Aplikacja wykorzystuje bazę danych `ROOM` do przechowywania wszystkich zadań, umożliwia podstawowe operacje *CRUD* (*Create-Remove-Update-Delete*), oraz ich filtrowanie i sortowanie.

<table><tr><td><img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExYWwxOGlnNGQ3Mm9xOGRraHFuaXFyOXA0dXkwNWI2OGx0M2RvbndlMSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c4MVddvPAaRxrwp9JB/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMnVzODY5dDR1a2QxdDk0MmtibWhmZXM0ZW0yc2VkcWZwd3NkaHlmZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/t3ERZ9ZxBVMeNiY1KY/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExbW8yN201Y3EzOHN6MGZsenhlMTM1ZHU0dWY5bmtkd2lvcTVybnZyZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/bCYIyjvwkKXnvrGd3n/giphy.gif" width="200" /></td></tr></table>

- Aplikacja zaimplementowana zgodnie ze wzoccem MVVM wraz z repozytorium
- `StateFlow` został wykorzystany do łatwego zarządzania bieżącym stanem interfejsu użytkownika
- Baza danych `Room` została wykorzystana do przechowywania zadań, oraz jako *SSOT* (*Single Source Of Truth*)
- Aplikacja zawiera trzy ekrany 
    - na głównym ekranie wyświetla listę zadań, umożliwia filtrowanie, sortowanie, usunięcie, oznaczenie zadania jako wykonanego, oraz szybką zmianę nadanego priorytetu
    - ekran dodania zadania
    - ekran aktualizacji zadania, który umożliwia również usunięcie
- Aplikacja wykorzystuje `Compose Navigation` w celu utworzenia nawigacji w aplikacji
- `ViewModel` został umieszczony w komponencie `Navigation` w celu uzyskania współdzielenia jego instancji przez wszystkie ekrany


Dodajmy wymagane zależności do projektu

```kotlin
    implementation ("androidx.lifecycle:lifecycle-viewmodel-compose:2.6.1")
    implementation ("androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1")
    implementation ("androidx.lifecycle:lifecycle-runtime-compose:2.6.1")

    implementation ("androidx.room:room-runtime:2.5.2")
    annotationProcessor ("androidx.room:room-compiler:2.5.2")
    kapt ("androidx.room:room-compiler:2.5.2")
    implementation("androidx.room:room-ktx:2.5.2")

    implementation ("androidx.navigation:navigation-compose:2.6.0")

    implementation ("androidx.compose.material:material-icons-extended:1.4.3")
```

W pierwszym kroku przygotujmy dane, które wykorzystamy jako *dummy data* do inicjalizacji bazy danych.

In [ ]:
data class Task(val title: String, val description: String)

In [ ]:
object DataProvider {

    private val titles = listOf(
        "Finish Report",
        "Prepare Presentation",
        "Call Client",
        "Update Website Content",
        "Attend Team Meeting",
        "Review Code",
        "Send Invoices",
        "Research Market Trends",
        "Plan Marketing Campaign",
        "Organize Files",
        "Prepare Budget Proposal",
        "Create Product Mockups",
        "Schedule Social Media Posts",
        "Meet with Project Team",
        "Test New Software",
        "Follow Up on Leads",
        "Design Logo",
        "Write Blog Post",
        "Arrange Project Timeline",
        "Review Product Specifications",
        "Arrange Travel Itinerary",
        "Conduct User Survey",
        "Analyze Sales Data",
        "Prepare Training Materials",
        "Coordinate Event Logistics",
        "Brainstorm New Ideas",
        "Optimize Website SEO",
        "Conduct Performance Reviews",
        "Develop App Prototypes"
    )

    private val descriptions = listOf(
        "Complete the final report for the quarterly performance.",
        "Create a PowerPoint presentation for the upcoming client meeting.",
        "Reach out to the client for a follow-up discussion.",
        "Update the content on the website's landing page.",
        "Coordinate logistics for an upcoming company event.",
        "Brainstorm innovative ideas for product improvement.",
        "Optimize the website's SEO for better search rankings.",
        "Conduct performance reviews for team members.",
        "Develop interactive prototypes for the new app.",
        "Test and evaluate the functionality of the new software thoroughly. Document any bugs or issues and work closely with the development team to ensure timely resolution.",
        "Follow up on potential leads and inquiries generated through marketing campaigns and networking events. Provide personalized responses and information tailored to each lead's needs.",
        "Design a new logo for the company rebranding, considering the brand's identity, target audience, and industry trends. Present multiple logo options for feedback and finalize the chosen design.",
        "Write a well-researched and informative blog post about recent industry developments. Use data, statistics, and expert insights to provide valuable content to the blog's readers.",
        "Arrange the timeline for the upcoming project phases, taking into account resource availability, dependencies, and potential risks. Share the timeline with the project stakeholders for feedback.",
        "Review and finalize the product specifications, ensuring they are comprehensive and align with the project's goals. Share the specifications with the development team for implementation.",
        "Plan the travel itinerary for the upcoming business trip, considering flight schedules, accommodation, transportation, and meeting arrangements. Share the itinerary with all relevant team members.",
        "Conduct a detailed survey to gather user feedback on the product's usability, features, and overall satisfaction. Analyze the survey results to identify areas for improvement.",
        "Analyze sales data and identify trends to make data-driven decisions for sales strategies and product improvements. Prepare a comprehensive report to present the findings to the sales team and management.",
        "Prepare training materials for new employees, including onboarding guides, training modules, and presentations. Ensure that the materials cover all essential aspects of the new hire's role.",
        "Coordinate logistics for an upcoming company event, such as a conference or team-building retreat. Secure venues, catering, transportation, and any necessary equipment.",
        "Brainstorm innovative ideas for product improvement, considering user feedback, market trends, and technological advancements. Present the ideas to the product development team for evaluation.",
        "Optimize the website's SEO to improve search engine rankings and organic traffic. Conduct keyword research, update meta tags, and implement on-page SEO best practices.",
        "Conduct performance reviews for team members, providing constructive feedback and setting goals for personal and professional growth. Recognize and reward exceptional performance.",
        "Develop interactive prototypes for the new app, incorporating user-friendly design and intuitive navigation. Test the prototypes with potential users for feedback and validation."
    )

    val tasks = (0..40).map { Task(title = titles.random(), description = descriptions.random()) }
}

## Layout

Rozpocznijmy od podstawowego layoutu ekranu głównego. Chcemy wyświetlić listę wszystkich zadań, ale zrobimy to w dwóch kolumnach - lista będzie wyświetlana niesymetrycznie. W tym celu wykorzystamy `LazyVerticalStaggeredGrid`, dzięki temu uzyskamy efekt siatki z przesuniętymi kafelkami (*staggered grid*). Jest to to układ wyświetlania elementów w interfejsie, w którym elementy mają różne wysokości lub szerokości, co powoduje, że są one *przesunięte* względem siebie. Dzięki temu można uzyskać efekt nieregularnego, dynamicznego układu, idealnego do prezentowania elementów o różnych rozmiarach. 

Pole `TextField` umożiwiające filtrowanie, oraz przecisk z menu kontekstowym służącym do filtrowania listy po priorytecie, umieścimy na belce górnej ekranu. Jako główny element wykorzystamy `Scaffold`. który zapewnia strukturę dla popularnych układów, umożliwiając programistom skupienie się na tworzeniu zawartości interfejsu, a nie na ręcznym zarządzaniu każdym elementem UI.

In [ ]:
Scaffold() { paddingValues ->                                     // wykorzystujemy aby części Scaffold nie pokrywały się z innymi elementami
    // jako zewnętrzny element listy dodajemy Box aby łatwiej ustawić marginesy wokół listy
    Box(
        modifier = Modifier
            .fillMaxSize()
            .padding(paddingValues)
            .padding(top = 12.dp)
    ) {
        LazyVerticalStaggeredGrid(                                // lista jako niesymetryczne kafelki
            columns = StaggeredGridCells.Adaptive(200.dp),        // każda kulumna ma no najmniej 200.dp
                                                                  // dynamiczna liczba komumn w zależności od wielkości ekranu
            verticalItemSpacing = 8.dp,                           // wertykalny margines między elementami
            horizontalArrangement = Arrangement.spacedBy(8.dp),   // odległość między kolumnami
            content = {                                           // zawartość listy
                items(DataProvider.tasks.size) { index ->         // tak jak w LazyColumn/LazyRow
                    CreateItemCard(                               // tworzyym kartę dla każdego elementu
                        onClickHigherPriority = { /*TODO()*/ },   // przekazane metody onClick
                        onClickDone = { /*TODO()*/ },
                        onClickLowerPriority = { /*TODO()*/ },
                        onClickEdit = { /*TODO()*/ },
                        task = tasks[index]                       // przekazany aktualne zadanie z listy
                    )
                }
            },
            modifier = Modifier.fillMaxSize()                     // wypełniamy całą dostępną przestrzeń
        )
    }
}

- `Scaffold() { paddingValues -> ... }` - Jest to główny komponent, który dostarcza strukturę dla interfejsu użytkownika. 
- `Box(...)` - kontener, który pozwala na komponowanie elementów w warstwach, umożliwiając dowolne ułożenie i komponowanie interfejsu. Wewnątrz tego Box znajduje się zawartość listy.
- `LazyVerticalStaggeredGrid(...)` - komponent, który tworzy pionową siatkę z przesuniętymi kafelkami. 
- `items(DataProvider.tasks.size) { index -> ... }` - Jest to funkcja, która tworzy elementy w siatce.
- `CreateItemCard(...)` - Jest to komponent, który jest wywoływany wewnątrz funkcji items i tworzy kartę dla każdego elementu.

In [ ]:
// główeny element listy
@Composable
private fun CreateItemCard(
    onClickHigherPriority: () -> Unit,
    onClickDone: () -> Unit,
    onClickLowerPriority: () -> Unit,
    onClickEdit: () -> Unit,
    task: Task
) {
    Card(
        shape = RoundedCornerShape(15.dp),
        elevation = CardDefaults.cardElevation(defaultElevation = 10.dp),
    ) {
        Column(
            modifier = Modifier.fillMaxSize()
        ) {
            CardTitle(task.title)                                            // tworzymy tytuł karty
            
            Text(                                                            // opis zadania
                text = task.description,
                modifier = Modifier.padding(4.dp),
                fontSize = 14.sp
            )

            // każda karta będzie zawierała cztery przyciski - 
            // - zwiększający/zmniejszający priorytet zadania, 
            // - oznaczający zadanie jako wykonane, 
            // - umożliwiający edycję zadania
            ButtonsRow(                                                      // rząd przycisków (dolne menu) każdej karty
                onClickHigherPriority = onClickHigherPriority,
                onClickDone = onClickDone,
                onClickLowerPriority = onClickLowerPriority,
                onClickEdit = onClickEdit
            )

        }
    }
}

- `@Composable private fun CreateItemCard(...)` - tworzy kartę reprezentującą zadanie. Przyjmuje jako argumenty kilka funkcji obsługujących różne akcje, takie jak zwiększanie/obniżanie priorytetu, oznaczenie zadania jako wykonane, edycja zadania, oraz obiekt typu `Task`, który reprezentuje konkretne zadanie.
- `Card(...)` - Tworzy komponent karty wizualnie reprezentującej zadanie. `shape` ustawia zaokrąglone rogi karty, a `elevation` ustawia cień karty.
- `CardTitle(task.title)` - Tworzy tytuł karty, przekazuje tytuł zadania jako argument do komponentu.
- `Text(...)` - Tworzy opis zadania w formie tekstu.
- `ButtonsRow(...)` - Tworzy rząd przycisków reprezentujących akcje związane z zadaniem, takie jak zmiana priorytetu, oznaczenie jako wykonane, edycja itp.

In [ ]:
// tytuł każdej karty
@Composable
private fun CardTitle(title: String) {
    Box(
        modifier = Modifier
            .fillMaxWidth()
            .background(Color(147, 176, 230))                   // zmieniamy kolor tła

    ) {
        Row(
            modifier = Modifier
                .fillMaxWidth(),
            verticalAlignment = Alignment.CenterVertically
        ) {
            Text(
                text = title,                                   // ustawiamy tytuł zadania
                fontWeight = FontWeight.Bold,
                style = MaterialTheme.typography.titleMedium,   // styl tytułu
                modifier = Modifier
                    .padding(4.dp)
                    .weight(1f),                                // zajmuje całą dostępną przestrzeń
                fontSize = 16.sp,
                textAlign = TextAlign.Center,                   // wyśridkowanie
            )

            CreatePriorityIcon(Color(128, 203, 196))            // na każdym pasku tytułu wyświetlamy 
                                                                // ikonę oznaczającą priorytet zadania
        }
    }
}

- `@Composable private fun CardTitle(title: String)` - Jest to komponent, który tworzy tytuł karty dla zadania.
- `Text(...)` - Tworzy tekst zawierający tytuł zadania.
- `modifier = Modifier.padding(4.dp).weight(1f)` - Dodaje odstęp do tekstu oraz ustawia wagę na `1f`. Waga `1f` oznacza, że komponent `Text` zajmuje całą dostępną szerokość, co pomaga wycentrować tytuł.
- `CreatePriorityIcon(Color(128, 203, 196))` - Wywołuje komponent, który tworzy ikonę reprezentującą priorytet zadania.

In [ ]:
// ikona oznaczająca priorytet zadania
@Composable
private fun CreatePriorityIcon(color: Color) {
    Box(
        modifier = Modifier
            .fillMaxHeight()
            .padding(end = 4.dp)
    )
    {
        Card(
            elevation = CardDefaults.cardElevation(10.dp),
            colors = CardDefaults.cardColors(
                containerColor = Color(0xFF6650a4)
            )
        ) {
            Image(                                       // ikona
                imageVector = Icons.Default.Circle,      // grafika
                contentDescription = "Priority",
                colorFilter = ColorFilter.tint(color)    // zmiana koloru ikony
            )
        }
    }
}

- `@Composable private fun CreatePriorityIcon(color: Color)` - Jest to komponent, który tworzy ikonę priorytetu dla zadania. Przyjmuje kolor ikony jako argument.
- `Card(...)` - Tworzy komponent karty, który otacza ikonę.
- `Image(...)` - Tworzy obrazek (ikona) reprezentujący priorytet. Ustawia grafikę jako ikonę okręgu (`Icons.Default.Circle`). `colorFilter` zmienia kolor ikony na podany kolor, który został przekazany do komponentu jako argument.

In [ ]:
// przyciski karty
@Composable
private fun ButtonsRow(
    onClickHigherPriority: () -> Unit,
    onClickDone: () -> Unit,
    onClickLowerPriority: () -> Unit,
    onClickEdit: () -> Unit
) {
    Row(
        modifier = Modifier
            .fillMaxWidth()
            .padding(top = 8.dp),
        Arrangement.SpaceEvenly,
        verticalAlignment = Alignment.CenterVertically
    ) {
        CreateOutlinedIconButton(
            onClick = onClickHigherPriority,
            icon = Icons.Outlined.KeyboardArrowUp
        )
        CreateOutlinedIconButton(
            onClick = onClickDone,
            icon = Icons.Outlined.Done
        )
        CreateOutlinedIconButton(
            onClick = onClickLowerPriority,
            icon = Icons.Outlined.KeyboardArrowDown
        )
    }
}

- `@Composable private fun ButtonsRow(...)` - Jest to komponent, który tworzy rząd przycisków dla akcji związanych z zadaniem. Przyjmuje jako argumenty funkcje obsługujące różne akcje: zwiększanie priorytetu, oznaczenie jako wykonane, obniżanie priorytetu i edycję.
- `Row(...)` - Tworzy rząd przycisków, które będą wyświetlane obok siebie.
- `Arrangement.SpaceEvenly` - Określa równomierne rozmieszczenie przycisków wzdłuż osi poziomej.
- `verticalAlignment = Alignment.CenterVertically` - Wyrównuje przyciski w pionie, aby były wyśrodkowane względem siebie.
- `CreateOutlinedIconButton(...)` - Wywołuje komponent CreateOutlinedIconButton, który tworzy przycisk z ikoną.

In [ ]:
@Composable
private fun CreateOutlinedIconButton(
    onClick: () -> Unit,
    icon: ImageVector
) {
    OutlinedIconButton(
        onClick = onClick,
        enabled = true,
        modifier = Modifier.size(40.dp),
        border = BorderStroke(2.dp, Color(0xFF6650a4)),
        colors = IconButtonDefaults.iconButtonColors(
            contentColor = Color(0xFF6650a4),
        ),
        content = {
            Icon(
                imageVector = icon,
                contentDescription = "Lower priority"
            )
        }
    )
}

- `OutlinedIconButton(...)` - Tworzy przycisk z ikoną w konturze. Ustawia funkcję obsługującą kliknięcie przycisku (`onClick`).

W efekcie dostajemy

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExNjByenhpeDllNjJnaTV5amM4M2ZrNTBvNmE0dmZydDRxeDRvYmt4OSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/noe3gCVDVysM5RNOfM/giphy.gif" width="200" />

Uzupełnijmy nasz ekrna główny o `FAB` (*Floating Action Button*) oraz dodajmy `topBar` do elementu `Scaffold`

In [ ]:
@OptIn(ExperimentalFoundationApi::class, ExperimentalMaterial3Api::class)
@Composable
fun TaskListScreen() {
    Scaffold(
        // dodajemy FAB
        floatingActionButton = {
            AddTaskFAB(onClick = { /* TODO */ })       // funkcja tworząca przycisk, do której przekazujemy onClick
        },
        topBar = {
            TopAppBar(
                title = { FilteringEditText() },       // Funkcja tworząca pole title górnej belki
                actions = { PrioritySorting() },       // Funkcja dodająca opcjonalną akcję po prawej stronie od tytułu
                modifier = Modifier.height(60.dp)
            )
        }
    ) { paddingValues ->
        ...
    }
}

- `floatingActionButton` - dodaje przycisk `FAB` na ekranie, który tworzony jest za pomocą komponentu `AddTaskFAB`.
- `topBar = { ... }` - Tworzy pasek aplikacji na górze ekranu. `title` wykorzystuje komponent `FilteringEditText`, który tworzy pole tekstowe do filtrowania zadań. `actions` wykorzystuje komponent `PrioritySorting`, który dodaje opcjonalną akcję sortowania po prawej stronie od tytułu.

In [ ]:
// FAB
@Composable
private fun AddTaskFAB(
    onClick: () -> Unit
) {
    FloatingActionButton(
        onClick = onClick,
        elevation = FloatingActionButtonDefaults.bottomAppBarFabElevation(10.dp),
        containerColor = Color(100, 149, 237),
        shape = FloatingActionButtonDefaults.smallShape
    ) {
        Icon(
            imageVector = Icons.Filled.Add,
            contentDescription = "Add item"
        )
    }
}

- `@Composable private fun AddTaskFAB(...)` -  Jest to komponent, który tworzy przycisk `FAB` do dodawania nowego zadania. Przyjmuje jako argument funkcję obsługującą kliknięcie przycisku.
- `FloatingActionButton(...)` - Tworzy przycisk `FAB` w interfejsie. `onClick` ustawia funkcję obsługującą kliknięcie przycisku. `elevation` ustawia wysokość cienia przycisku. `containerColor` ustawia kolor tła przycisku na podany kolor. `shape` ustawia kształt przycisku na mały za pomocą `FloatingActionButtonDefaults.smallShape`.
- `Icon(...)` - Tworzy ikonę reprezentującą dodawanie nowego zadania.

In [ ]:
@Composable
@OptIn(ExperimentalMaterial3Api::class)
private fun FilteringEditText() {
    TextField(
        value = "",
        singleLine = true,                                      // pole wyświetla pojedynczą linię tekstu
        modifier = Modifier.fillMaxWidth(),
        colors = TextFieldDefaults.textFieldColors(
            containerColor = MaterialTheme.colorScheme.surface
        ),
        onValueChange = { /* TODO */ },
        label = { Text(text = "Filtruj") },
        textStyle = TextStyle.Default.copy(fontSize = 20.sp)    // zmieniamy wielkość czcionki
    )
}

- `TextField(...)` - Tworzy pole tekstowe w interfejsie. `value` ustawia aktualną wartość pola tekstowego. `singleLine = true` informuje, że pole tekstowe ma wyświetlać tylko jedną linię tekstu.
- `onValueChange = { /* TODO */ }` - Jest to funkcja obsługująca zmianę wartości w polu tekstowym.
- `label = { Text(text = "Filtruj") }` - Tworzy etykietę pola tekstowego, która wyświetla się nad polem tekstowym.

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExdGFndDAwNGJya2kwc3hscjc5ZzdsN3FtMGw1dXBkbTNhMGFzOGdyOCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/taKjg0CDkR0mCfarqk/giphy.gif" width="200" />

Dodajmy jeszcze możliwość usuwania zadania do `ButtonsRow`

In [ ]:
@Composable
private fun ButtonsRow(
    onClickHigherPriority: () -> Unit,
    onClickDone: () -> Unit,
    onClickLowerPriority: () -> Unit,
    onClickEdit: () -> Unit
) {
    Column {
        Row(
            modifier = Modifier
                .fillMaxWidth()
                .padding(top = 8.dp),
            Arrangement.SpaceEvenly,
            verticalAlignment = Alignment.CenterVertically
        ) {
            CreateOutlinedIconButton(
                onClick = onClickHigherPriority,
                icon = Icons.Outlined.KeyboardArrowUp
            )
            CreateOutlinedIconButton(
                onClick = onClickDone,
                icon = Icons.Outlined.Done
            )
            CreateOutlinedIconButton(
                onClick = onClickLowerPriority,
                icon = Icons.Outlined.KeyboardArrowDown
            )

            OutlinedIconButton(
                onClick = { /* doSomething() */ },
                shape = FloatingActionButtonDefaults.smallShape,
                modifier = Modifier.padding(start = 4.dp),
                border = BorderStroke(2.dp, Color(0xFF6650a4)),
                colors = IconButtonDefaults.iconButtonColors(
                    contentColor = Color(0xFF6650a4),
                ),
            ) {
                Icon(
                    imageVector = Icons.Filled.Delete,
                    contentDescription = "Delete",
                    tint = Color(0xAA90021F)
                )
            }
        }
        OutlinedIconButton(
            onClick = onClickEdit,
            enabled = true,
            modifier = Modifier
                .fillMaxWidth()
                .padding(start = 4.dp, end = 4.dp),
            border = BorderStroke(2.dp, Color(0xFF6650a4)),
            colors = IconButtonDefaults.iconButtonColors(
                contentColor = Color(0xFF6650a4),
            ),
            content = {
                Icon(
                    imageVector = Icons.Outlined.Edit,
                    contentDescription = "Edit task"
                )
            }
        )
    }
}

Dodajmy pozostałe dwa ekrany, ich layout jest relatywnie prosty.

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun AddTaskScreen(onHome: () -> Unit){
    
    val radioOptions = listOf("Low", "Normal", "High") // lista opcji dla RadioButton
    
    val (selectedOption, onOptionSelected) = remember { mutableStateOf(radioOptions[1] ) } // zapamiętujemy stan RadioButton
/*
    -selectedOption: Jest to zmienna, która przechowuje aktualnie wybraną opcję spośród dostępnych opcji.
    W tym przypadku, wartość selectedOption będzie początkowo ustawiona na drugą opcję z listy radioOptions.

    - onOptionSelected: Jest to funkcja, która będzie wywoływana, gdy użytkownik wybierze inną opcję.
    
    - remember: pozwala na trwałe zachowanie i automatyczne przetrzymywanie pewnych wartości między odświeżeniami interfejsu. 
    W przypadku zmiennej selectedOption, jest to użyteczne, aby stan wyboru opcji nie był tracony podczas zmiany stanu interfejsu.
    
    powyżej tworzymy zmienną selectedOption, która przechowuje wybraną opcję, oraz funkcję onOptionSelected, która pozwala na aktualizację tej opcji. 
    Dzięki remember i mutableStateOf, zmiany w selectedOption automatycznie spowodują aktualizację interfejsu w reakcji na nowy wybór.
*/
 
    val checkedState = remember { mutableStateOf(true) }   // zapamiętujemy stan chackBox
    
    Column(modifier = Modifier
        .fillMaxSize()
        .padding(8.dp)
    ) {
        // tytuł zadania
        OutlinedTextField(
            value = "",
            onValueChange = {},
            modifier = Modifier.fillMaxWidth(),
            singleLine = true,
            label = { Text(text = "Tytuł")}
        )

        // opis zadania
        OutlinedTextField(
            value = "",
            onValueChange = {},
            modifier = Modifier
                .fillMaxWidth()
                .fillMaxHeight(.5f),
            singleLine = false,
            label = { Text(text = "Opis")}
        )
        
        Row(
            modifier = Modifier.fillMaxWidth().padding(top = 8.dp),
        ) {
            OutlinedCard(
                modifier = Modifier.fillMaxWidth(.5f).padding(end = 2.dp),
            ) {
                Column(
                    verticalArrangement = Arrangement.SpaceEvenly,
                    horizontalAlignment = Alignment.CenterHorizontally
                ) {
                    // ustawienie priorytetu
                    radioOptions.forEach { text ->
                        Row(
                            Modifier
                                .fillMaxWidth()
                                .selectable( // obiekt jest zaznaczalny
                                    selected = (text == selectedOption), // obsługa zaznaczenia
                                    onClick = { onOptionSelected(text) } // obsługa onclick zaznaczenia
                                )
                                .padding(horizontal = 16.dp),
                            verticalAlignment = Alignment.CenterVertically
                        ) {
                            RadioButton(
                                selected = (text == selectedOption),
                                onClick = { onOptionSelected(text) }
                            )
                            Text(
                                text = text,
                                style = MaterialTheme.typography.bodyMedium.merge(),
                                modifier = Modifier.padding(start = 16.dp),
                                fontSize = 24.sp
                            )
                        }
                    }
                }
            }

            OutlinedCard(
                modifier = Modifier
                    .fillMaxWidth()
                    .height(144.dp)
                    .padding(start = 2.dp)
            ) {
                // Chekbox pozwalający ustawić zmienną oznaczającą wykonanie zadania
                Column(
                    modifier = Modifier.fillMaxSize(),
                    horizontalAlignment = Alignment.CenterHorizontally,
                    verticalArrangement = Arrangement.Center
                ) {
                    Text(
                        text = "Czy zakończone",
                        fontSize = 24.sp,
                        modifier = Modifier.padding(start = 4.dp)
                    )

                    Checkbox(
                        modifier = Modifier.scale(1.5f).padding(top = 8.dp),
                        checked = checkedState.value,
                        onCheckedChange = { checkedState.value = it }
                    )
                }
            }
        }
        Row(
            modifier = Modifier.fillMaxWidth().padding(top = 8.dp)
        ) {

            // przyciski zapisu do bazy i możliwośc powrotu do poprzedniego ekranu bez zapisu
            Button(
                onClick = onHome,
                modifier = Modifier.fillMaxWidth(.5f)
            ) {
                Text(
                    text = "Zapisz",
                    fontSize = 20.sp
                )
            }
            OutlinedButton(
                onClick = onHome,
                modifier = Modifier.fillMaxWidth()
            ) {
                Text(
                    text = "Wróć",
                    fontSize = 20.sp
                )
            }
        }
    }
}

- `val radioOptions = listOf("Low", "Normal", "High")` - Tworzy listę opcji dla pól typu RadioButton.
- `val (selectedOption, onOptionSelected) = remember { mutableStateOf(radioOptions[1] ) }` - Jest to mechanizm, który zapamiętuje stan wyboru opcji RadioButton. `selectedOption` przechowuje aktualnie wybraną opcję. `onOptionSelected` to funkcja, która jest wywoływana, gdy użytkownik wybierze inną opcję.
- `val checkedState = remember { mutableStateOf(true) }` - Zapamiętuje stan zaznaczenia dla CheckBox.
- `OutlinedTextField(...)` - Tworzy pole tekstowe typu OutlinedTextField do wprowadzania tytułu i opisu zadania.
- `OutlinedCard(...)` - Tworzy kartę typu OutlinedCard, która jest używana do grupowania zawartości. Zawiera sekcję ustawiania priorytetu za pomocą pól typu `RadioButton`, oraz sekcję z `CheckBox`.

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun UpdateTaskScreen(onHome: () -> Unit){
    
    val radioOptions = listOf("Low", "Normal", "High")
    val (selectedOption, onOptionSelected) = remember { mutableStateOf(radioOptions[1] ) }
    val checkedState = remember { mutableStateOf(true) }
    
    Column(modifier = Modifier
        .fillMaxSize()
        .padding(8.dp)
    ) {
        OutlinedTextField(
            value = "",
            onValueChange = {},
            modifier = Modifier.fillMaxWidth(),
            singleLine = true,
            label = { Text(text = "Tytuł")}
        )

        OutlinedTextField(
            value = "",
            onValueChange = {},
            modifier = Modifier
                .fillMaxWidth()
                .fillMaxHeight(.5f),
            singleLine = false,
            label = { Text(text = "Opis")}
        )
        
        Row(
            modifier = Modifier
                .fillMaxWidth()
                .padding(top = 8.dp),
        ) {
            OutlinedCard(
                modifier = Modifier
                    .fillMaxWidth(.5f)
                    .padding(end = 2.dp),
            ) {
                Column(
                    verticalArrangement = Arrangement.SpaceEvenly,
                    horizontalAlignment = Alignment.CenterHorizontally
                ) {
                    radioOptions.forEach { text ->
                        Row(
                            Modifier
                                .fillMaxWidth()
                                .selectable(
                                    selected = (text == selectedOption),
                                    onClick = {
                                        onOptionSelected(text)
                                    }
                                )
                                .padding(horizontal = 16.dp),
                            verticalAlignment = Alignment.CenterVertically
                        ) {
                            RadioButton(
                                selected = (text == selectedOption),
                                onClick = { onOptionSelected(text) }
                            )
                            Text(
                                text = text,
                                style = MaterialTheme.typography.bodyMedium.merge(),
                                modifier = Modifier.padding(start = 16.dp),
                                fontSize = 24.sp
                            )
                        }
                    }
                }
            }

            OutlinedCard(
                modifier = Modifier
                    .fillMaxWidth()
                    .height(144.dp)
                    .padding(start = 2.dp)
            ) {
                Column(
                    modifier = Modifier.fillMaxSize(),
                    horizontalAlignment = Alignment.CenterHorizontally,
                    verticalArrangement = Arrangement.Center
                ) {
                    Text(
                        text = "Czy zakończone",
                        fontSize = 24.sp,
                        modifier = Modifier.padding(start = 4.dp)
                    )

                    Checkbox(
                        modifier = Modifier
                            .scale(1.5f)
                            .padding(top = 8.dp),
                        checked = checkedState.value,
                        onCheckedChange = { checkedState.value = it }
                    )
                }
            }
        }
        Row(
            modifier = Modifier
                .fillMaxWidth()
                .padding(top = 8.dp)
        ) {


            // przyciski umożliwiające aktualizację, usunięcie zadania oraz powrót do poprzedniego ekranu
            Button(
                onClick = onHome,
                modifier = Modifier.fillMaxWidth(.5f).padding(end = 2.dp)
            ) {
                Text(
                    text = "Aktualizuj",
                    fontSize = 20.sp
                )
            }
            Button(
                onClick = onHome,
                modifier = Modifier.fillMaxWidth().padding(start = 2.dp)
            ) {
                Text(
                    text = "Usuń",
                    fontSize = 20.sp
                )
            }
        }
        OutlinedButton(
            onClick = onHome,
            modifier = Modifier.fillMaxWidth()
        ) {
            Text(
                text = "Wróć",
                fontSize = 20.sp
            )
        }
    }
}

Następnie przejdźmy do nawigacji. zdefiniujmy hierarchię ekranów z których składa się aplikacja.

In [ ]:
sealed class Screens(
    val route: String
) {
    object TaskList : Screens("tasklist")
    object Add : Screens("add")
    object Update : Screens("update")
}

Dodajmy samą nawigację.

In [ ]:
@Composable
fun Navigation() {
    val navController = rememberNavController()

    NavHost(navController = navController, startDestination = Screens.TaskList.route) {
        composable(route = Screens.TaskList.route){
            TaskListScreen (
                onAddScreen = {navController.navigate(Screens.Add.route) },
                onEditScreen = { navController.navigate(Screens.Update.route) }
            )
        }

        composable(route = Screens.Add.route){
            AddTaskScreen { navController.popBackStack() }
        }

        composable(route = Screens.Update.route){
            UpdateTaskScreen { navController.popBackStack() }
        }
    }
}

<img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExNWZnZzZsNjZlYjhoZWYyamN0bTR4b3Y0M2k2dnlkYjczdWxsNGRwbCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/2jrNlBM6JzHt06HSZ0/giphy.gif" width="200" />

## Baza danych

Rozpocznijmy od dodania `Entity`

In [ ]:
@Entity(tableName = "task_table")
data class Task(
    @PrimaryKey(autoGenerate = true) val id: Int = 0,
    val title: String,
    val description: String,
    val isDone: Boolean = false,
    val priority: Priority = Priority.MEDIUM
)

Priorytet dodamy jako klasę `enum`

In [ ]:
enum class Priority {
    HIGH,
    MEDIUM,
    LOW,
    NONE
}

W kolejnym kroku określmy metody które będą nam potrzebne w `dao`

In [ ]:
@Dao
interface TaskDao {

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insertTask(task: Task)

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insertAllTasks(tasks: List<Task>)

    @Query("SELECT * FROM task_table ORDER BY id ASC")
    fun getTasks(): Flow<List<Task>>

    @Delete
    suspend fun deleteTask(task: Task)

    @Update
    suspend fun updateTask(task: Task)

    @Query("SELECT * FROM task_table WHERE title LIKE :title")
    fun getFilteredTasksByTitle(title: String): Flow<List<Task>>

    @Query("DELETE FROM task_table")
    suspend fun deleteAll()

    @Query("SELECT * FROM task_table WHERE priority = :priority")
    fun getFilteredTasksByPriority(priority: String): Flow<List<Task>>
}

- `@Query("SELECT * FROM task_table ORDER BY id ASC")` - wykonuje zapytanie SQL do pobrania wszystkich zadań z tabeli `task_table`. `ORDER BY id ASC` sortuje wyniki wg rosnącego klucza głównego.
- `@Delete` - usuwa zadanie z bazy danych.
- ` @Update` - aktualizuje istniejące zadanie w bazie danych.
- `@Query("SELECT * FROM task_table WHERE title LIKE :title")` - wykonuje zapytanie SQL do pobrania zadań z tabeli `task_table` na podstawie tytułu. `LIKE :title` pozwala na wykonanie wyszukiwania częściowego na podstawie tytułu.
- `@Query("DELETE FROM task_table")` - usuwa wszystkie zadania z tabeli task_table.
- `@Query("SELECT * FROM task_table WHERE priority = :priority")` - wykonuje zapytanie SQL do pobrania zadań z tabeli `task_table` na podstawie priorytetu.

Kolejnym krokiem będzie dodanie repozytorium.

In [ ]:
class TaskRepository(private val application: Application) {
    private val db = TaskDatabase.getDatabase(application)
    private val dao = db.taskDao()

    fun getTasks() = dao.getTasks()
    fun getFilteredTasksByTitle(title: String) = dao.getFilteredTasksByTitle(title)
    fun getFilteredTasksByPriority(priority: String) = dao.getFilteredTasksByPriority(priority)

    suspend fun insertTask(task: Task) = dao.insertTask(task)
    suspend fun insertAllTasks(tasks: List<Task>) = dao.insertAllTasks(tasks)
    suspend fun deleteTask(task: Task) = dao.deleteTask(task)
    suspend fun updateTask(task: Task) = dao.updateTask(task)
    suspend fun deleteAll() = dao.deleteAll()
}

Ponieważ `Room` wymaga podania kontekstu, musimy zaimplementować fabrykę dla `ViewModel`

In [ ]:
class TaskViewModelFactory(private val application: Application) :
    ViewModelProvider.Factory {
    override fun <T : ViewModel> create(modelClass: Class<T>): T {
        return TaskViewModel(application) as T
    }
}

Przejdźmy do dodania samego `ViewModel`

In [ ]:
class TaskViewModel(application: Application) : ViewModel() {

    private val repository = TaskRepository(application)

    private val _tasksState = MutableStateFlow<List<Task>>(emptyList())
    val tasksState: StateFlow<List<Task>>
        get() = _tasksState

    init {
        reinitializeDatabaseWithDummyData()
        getAll()
    }

    private fun reinitializeDatabaseWithDummyData(){
        deleteAll()
        addAll(DataProvider.tasks)
    }

    private fun addAll(tasks: List<Task>){
        viewModelScope.launch {
            repository.insertAllTasks(tasks)
        }
    }

    private fun deleteAll(){
        viewModelScope.launch {
            repository.deleteAll()
        }
    }

    private fun getAll(){
        viewModelScope.launch {
            repository.getTasks().collect{tasks ->
                _tasksState.value = tasks
            }
        }
    }

    fun filterByTitle(title: String){
        viewModelScope.launch {
            repository.getFilteredTasksByTitle("%$title%").collect {filteredList ->
                _tasksState.value = filteredList
            }
        }
    }
}

- `private val repository = TaskRepository(application)` -  Inicjalizuje obiekt `repository`, która obsługuje operacje bazodanowe.
- `_tasksState` i `tasksState` - `_tasksState` to prywatna zmienna, która przechowuje aktualny stan listy zadań. `tasksState` to publiczny getter dla `_tasksState`, który umożliwia dostęp do stanu listy zadań jako `StateFlow`.
- `init { ... }` - Blok init zawiera kod, który jest wykonywany podczas tworzenia instancji `TaskViewModel`. Wywołuje funkcje do ponownego inicjalizowania bazy danych z danymi testowymi oraz pobierania wszystkich zadań.
- `reinitializeDatabaseWithDummyData() { ... }` - Funkcja, która usuwa wszystkie istniejące zadania i dodaje dane testowe do bazy danych.
- `addAll(tasks: List<Task>) { ... }` - Funkcja, która dodaje listę zadań do bazy danych.
- `deleteAll() { ... }` - Funkcja, która usuwa wszystkie zadania z bazy danych.
- `getAll() { ... }` - Funkcja, która pobiera wszystkie zadania z bazy danych i aktualizuje `_tasksState`.
- `filterByTitle(title: String) { ... }` - Funkcja, która pobiera zadania z filtrowaniem po tytule.

N tym etapie zaimplementujemy tylko filtrowanie.

`ViewModel` zainicjujemy w komponencie nawigacji, pozwala to na udostępnianie stanu danych między różnymi ekranami aplikacji. Dzięki temu, stan i logika związana z danymi mogą być współdzielone i zarządzane w jednym miejscu.

In [ ]:
@Composable
fun Navigation() {
    val navController = rememberNavController()
    val viewModel: TaskViewModel = viewModel(
        LocalViewModelStoreOwner.current!!,
        "TaskViewModel",
        TaskViewModelFactory(LocalContext.current.applicationContext as Application)
    )

    NavHost(navController = navController, startDestination = Screens.TaskList.route) {
        composable(route = Screens.TaskList.route){
            TaskListScreen (
                onAddScreen = {navController.navigate(Screens.Add.route) },
                onEditScreen = { navController.navigate(Screens.Update.route) },
                viewModel = viewModel
            )
        }

        composable(route = Screens.Add.route){
            AddTaskScreen { navController.popBackStack() }
        }

        composable(route = Screens.Update.route){
            UpdateTaskScreen { navController.popBackStack() }
        }
    }
}

`ViewModel` przekazujemy jako argument do ekranów, teraz dodajmy tylko na ekran główny, aby zaimplementować filtrowanie.

W komponencie `TaskListScreen` dodamy obsługę filtrowania, oraz zmodyfikujemy listę zadań, aby była pobierana z bazy danych a nie bezpośrednio z `DataProvider`

In [ ]:
@OptIn(ExperimentalFoundationApi::class, ExperimentalMaterial3Api::class)
@Composable
fun TaskListScreen(
    onAddScreen: () -> Unit,
    onEditScreen: () -> Unit,
    viewModel: TaskViewModel
) {
    val tasks by viewModel.tasksState.collectAsStateWithLifecycle()
    var filteredTitle by remember {
        mutableStateOf("")
    }
    
    Scaffold(
        floatingActionButton = {
            AddTaskFAB(onClick = { onAddScreen() })
        },
        topBar = {
            TopAppBar(
                title = {
                    FilteringEditText(filteredTitle) { title ->
                        filteredTitle = title
                        viewModel.filterByTitle(filteredTitle)
                    }
                },
                actions = { PrioritySorting() },
                modifier = Modifier.height(60.dp)
            )
        }
    ) {
        ...
            LazyVerticalStaggeredGrid(
                columns = StaggeredGridCells.Adaptive(200.dp),
                verticalItemSpacing = 8.dp,
                horizontalArrangement = Arrangement.spacedBy(8.dp),
                content = {
                    items(tasks.size) { index ->    // zmieniamy DataProvider.tasks.size ma tasks.size aby wykorzystać dane z bazy
                        CreateItemCard(
                            onClickHigherPriority = { /*TODO()*/ },
                            onClickDone = { /*TODO()*/ },
                            onClickLowerPriority = { /*TODO()*/ },
                            onClickEdit = onEditScreen,
                            task = tasks[index]
                        )
                    }
                },
                modifier = Modifier.fillMaxSize()
            )
        ...
    }
}

- `var filteredTitle by remember { mutableStateOf("") }` - Tworzy zmienną, aby przechowywać wartość tytułu używanego do filtrowania zadań.
- `FilteringEditText(filteredTitle) { title -> ... }` - Komponent, który jest przekazywany jako tytuł. Akceptuje obecny filtrowany tytuł oraz funkcję zwrotną, która jest wywoływana po zmianie wartości filtrowanego tytułu. Wewnątrz funkcji zwrotnej aktualizuje wartość `filteredTitle` i wywołuje funkcję `filterByTitle` z viewModel w celu przefiltrowania zadań.

In [ ]:
@Composable
@OptIn(ExperimentalMaterial3Api::class)
private fun FilteringEditText(filteredTitle: String, onValueChange: (String) -> Unit) {
    TextField(
        value = filteredTitle,
        singleLine = true,
        modifier = Modifier.fillMaxWidth(),
        colors = TextFieldDefaults.textFieldColors(
            containerColor = MaterialTheme.colorScheme.surface
        ),
        onValueChange = onValueChange,                     // wywołana przy każdej zmianie wartości tekstu
        label = { Text(text = "Filtruj") },
        textStyle = TextStyle.Default.copy(fontSize = 20.sp)
    )
}

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExMzZjdTJrOGhnNTk3dXVmNGtlN2x0NHhlemg0Z3ZtaWJ5eDc4OXdidCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/b0teHy4FiteohvcVK2/giphy.gif" width="200" />